# Documentation assistant

This notebook demonstrates a documentation assistant: Video-to-documentation conversion using Vertex AI

Converting videos-to-documentation involves three steps: 
1. Protocol finder: Select protocol which best captures the step being performed in the video
2. Video comparing to ground-of-truth protocol → lab documentation + errors in procedure
3. Analytics based on benchmark dataset: automatic comparison of errors found by documentation assistent vs actual errors

In this notebook, I will focus on the first step - the protocol finder.

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [1]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [2]:
import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [3]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [4]:
from vertexai.generative_models import Part
from vertexai.preview.generative_models import GenerativeModel

MODEL_ID = "gemini-1.5-pro-001"

# Following: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/context-caching/intro_context_caching_vertex_ai_sdk.ipynb

In [5]:
# Upload protocols to GCS

directory = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/"

list_protocol_gcs = [
    video_to_protocol.upload_video_to_gcs(Path(directory) / file, bucket, "protocol_finder_2")
    for file in os.listdir(directory) 
    if "protocol" in file.lower()
]

list_protocol_gcs

['gs://mannlab_videos/protocol_finder_2/QueueSamples_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/PlaceEvotips_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/DisconnectColumn_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/Diluting_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/TimsCalibration_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/ConnectingColumnSampleLine_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/UltraSourceToESIsource_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/Pipette_protocolCorrect.md',
 'gs://mannlab_videos/protocol_finder_2/ESIsourceToUltraSource_protocolCorrect_CapillaryPushedIn.md',
 'gs://mannlab_videos/protocol_finder_2/Evotip_protocolCorrect.md']

In [6]:
# Prepare protocols as model input

protocol_input = []
count = 1
for protocol in list_protocol_gcs:
    protocol_input.append("number: " + str(count))
    protocol_input.append("file name: " + os.path.basename(protocol))
    protocol_input.append(Part.from_uri(protocol, mime_type="text/md"))
    count += 1

In [ ]:
# Prepare video for model input
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/PlaceEvotips_docuLiquidNotChecked.MP4"
video_uri = video_to_protocol.upload_video_to_gcs(video_path, bucket, "protocol_finder_2")

video_input = [
    "Lab video:",
    Part.from_uri(video_uri, mime_type="video/mp4"),
]

In [104]:
# Prompt

inputs = [
    """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video. 
    Protocols:
    """
]

inputs.extend(protocol_input)
inputs.extend(video_input)

inputs.append("Output format: Number of protocol")

In [105]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

model = GenerativeModel("gemini-1.5-flash-002")
response = model.generate_content(inputs, generation_config={"temperature": 0.9})
observation = response.text
print(response.usage_metadata)
print(video_path)
Markdown(observation)


prompt_token_count: 17655
candidates_token_count: 27
total_token_count: 17682
prompt_tokens_details {
  modality: TEXT
  token_count: 7110
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 10545
}
candidates_tokens_details {
  modality: TEXT
  token_count: 27
}

/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/PlaceEvotips_docuLiquidNotChecked.MP4


The video shows the step of spinning Evotips in a centrifuge.  Therefore, protocol number **10** is the correct one.

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from IPython.display import Markdown

def identify_protocol_from_video(
    model_input,
    model_name="gemini-1.5-flash-002", 
    temperature=0.9
):
    """Identifies the protocol number from a lab video using Gemini model with customizable prompts.
    
    Parameters
    ----------
    model_input : object
        Input to be processed by the model
    model_name : str
        Name of the Gemini model to use
    temperature : float
        Temperature for model generation
    
    Returns
    -------
    dict
        Dictionary containing the model response and metadata including:
        - observation: The text response from the model
        - usage_metadata: Usage statistics from the model
        - markdown_observation: Markdown formatted observation
    """
    model = GenerativeModel(model_name)
    response = model.generate_content(
        model_input, 
        generation_config={"temperature": temperature}
    )
    observation = response.text
    
    return {
        "observation": observation,
        "usage_metadata": response.usage_metadata,
        "video_path": video_path,
        "markdown_observation": Markdown(observation)
    }

In [ ]:
# Evaluation if correct

import re
import os
import pandas as pd

def validate_protocol_number(video_path, csv_path, observation):
    """Validates if the protocol number in the observation matches the expected protocol number from the benchmark dataset (csv).
    
    Parameters
    ----------
    video_path : str
        Path to the video file
    csv_path : str
        Path to the benchmark dataset as csv file containing protocol information
    observation : str
        Text containing protocol number to be extracted
    
    Returns
    -------
    dict
        Result of the validation containing match status and relevant details including:
        - matches: Boolean indicating if protocol numbers match
        - video: Basename of the video file
        - expected_protocol: Protocol number from CSV
        - found_protocol: Protocol number extracted from observation
        - message: Optional explanation message (only present on failure)
    """
    video_basename = os.path.basename(video_path)
    
    df_link = pd.read_csv(csv_path, delimiter=";")
    df_link = df_link.dropna()
    
    if video_basename not in list(df_link["documentation video"]):
        return {
            "matches": False,
            "video": video_basename,
            "expected_protocol": [],
            "found_protocol": [],
            "message": f"Video not part of benchmark dataset: {video_basename}"
        }
    
    expected_protocol_number = str(df_link['Number'][df_link["documentation video"]==video_basename].item())
    
    protocol_match = re.search(r'[Pp]rotocol number:?\s*(?:\*\*)?(\d+)(?:\*\*)?', observation)
    
    if not protocol_match:
        return {
            "matches": False,
            "video": video_basename,
            "expected_protocol": expected_protocol_number,
            "found_protocol": None,
            "message": "No protocol number found in observation"
        }
    
    found_protocol_number = protocol_match.group(1)
    
    return {
        "matches": expected_protocol_number == found_protocol_number,
        "video": video_basename,
        "expected_protocol": expected_protocol_number,
        "found_protocol": found_protocol_number
    }
    

In [ ]:
# First upload videos to Google Cloud Storage and convert to model input (reusable info, time consuming step)

video_directory = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation"

video_parts = {}

for video in os.listdir(video_directory):
    print(video)
    video_path = Path(video_directory) / video
    
    if video.lower().endswith(('.mp4', '.mov')):
        file_extension = os.path.splitext(video_path)[1].lower()[1:]
        
        video_uri = video_to_protocol.upload_video_to_gcs(video_path, bucket, "protocol_finder_2")

        video_parts[video] = {
            "path": str(video_path),
            "gcs_uri": video_uri,
            "part": [
                "Lab video:",
                Part.from_uri(video_uri, mime_type=f"video/{file_extension}")
            ]
        }

UltraSourceToESIsource_docuForgotN2Line.MP4
.DS_Store
TimsCalibration_docuCorrect_camera.MP4
TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
PlaceEvotips_docuBoxAngeled.MP4
UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap.MP4
QueueSamples_docuCorrect_1.mov
QueueSamples_docuNoBlankNoSampleIDWrongMSmethod.mov
UltraSourceToESIsource_docuCorrect.MP4
DisconnectingColumn_docuWithoutStandby_1.MP4
QueueSamples_docuDiffSampleNameAndEvotipNotAtPosition.mov
Pipette_ducoBufferBInsteadBufferA.MP4
ESIsourceToUltraSource_docuFogotOvenPowerSupply.MP4
Pipette_docu_WrongRowB_B10Twice_B11LeftOut.MP4
ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4
TimsCalibration_docuSavedMethod.mov
Dilute_docuWrongSolvent.MP4
TimsCalibration_docuWrongOrderSteps.mov
Dilute_docuWrongVolume_PipettTipNotChanged.MP4
DisconnectingColumn_docuCorrect.MP4
Pipette_docuFirstTwoWrongVolumneANDA2Twice.MP4
PlaceEvotips_docuWrongPosition.MP4
TimsCalibration_docuCorrect.mov
QueueSamples_docuWrongRow_S3A1Twice.mo

In [ ]:
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video.
    """]
protocol_intro= ["Protocols:"]
output_format = ["Output: Number of protocol e.g. 'protocol number: 1'"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, protocol_intro, protocol_input, video_parts[video]["part"], output_format]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


The video shows the change of the source from ESI to UltraSource. The protocol number 9 is the right one.

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '9'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


Protocol number: 5

{'matches': True, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '5'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '10'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


Protocol number: 10

{'matches': False, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '10'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Protocol number: 10

{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 51.72%


In [18]:
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video.
    """]
protocol_intro= ["Protocols:"]
output_format = ["Output: Number of protocol e.g. 'protocol number: 1'"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, video_parts[video]["part"], protocol_intro, protocol_input, output_format ]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Protocol number: 6

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


Based on the video, the protocol being performed is **protocol number: 1**,  "Queue and measure samples in HyStar".  The video shows a computer screen displaying the HyStar software interface, which is used for queuing and controlling LC-MS experiments.  The chromatograms and spectral data shown are consistent with data acquired during a mass spectrometry-based analysis.  The various parameters being adjusted (e.g., scan range, cycle time) further support this identification.


{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '1'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


Protocol number: 2

{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Protocol number: 10

{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 65.52%


In [19]:
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
    Your task is to find the protocol that captures the step being performed in the video. 
    I will first provide you with the to analyze video and then with the selection of protocols.
    """]
protocol_intro= ["Protocols:"]
output_format = ["Output: Number of protocol e.g. 'protocol number: 1'"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, video_parts[video]["part"], protocol_intro, protocol_input, output_format ]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Based on my expertise in proteomics and mass spectrometry, after analyzing the video, the most relevant protocol is **protocol number: 6**,  "Connecting IonOpticks Column & sample line of Evosep".

The video clearly shows the process of connecting a sample line to a column, which is precisely what protocol 6 describes.  The other protocols deal with sample preparation, software operation, and source switching – none of which are the central focus of the visual information provided.


{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


Protocol number: 1

{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '1'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


Protocol number: 2

{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Protocol number: 10

{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 65.52%


In [20]:
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    Your task is to analyze a video showing a laboratory procedure and identify which protocol from a provided list accurately describes the step being performed.
    Before viewing the potential protocols, please describe in detail what you observe in the video. Include: equipment used, actions performed, materials handled, and any notable techniques. Be specific about the laboratory procedure being demonstrated.
    I will first provide you with the to analyze video and then with the selection of protocols.
    Finish your resonse with the requested output format. 
    Take your time with this analysis. 
    """]
protocol_intro= ["Protocols:"]
output_format = ["Output: Number of protocol e.g. 'protocol number: 1'"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]
    order = [system_prompt, video_parts[video]["part"], protocol_intro, protocol_input, output_format ]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Good day.  Professor Matthias Mann here.


The video depicts a laboratory procedure involving the connection and disconnection of components within a mass spectrometry workflow, specifically focusing on a Bruker timsTOF mass spectrometer coupled with an Evosep One liquid chromatography system.


The video begins with a view of the assembled system, showing the timsTOF (the black and blue mass spectrometer), the Evosep One (the orange and grey liquid chromatography system), and associated computer monitors displaying software interfaces (likely HyStar and timsControl).


The first part of the video shows the removal of a component from the timsTOF, which appears to be the nano-electrospray source. The person carefully disconnects a capillary tube and then physically removes the part. They then move to a separate bench, putting on gloves before cleaning the component.


The next step shows the replacement of the removed component (the nano-electrospray source) back into the timsTOF, which is now shown to be in standby mode. The person carefully lines up the component and ensures a proper connection.  The use of gloves and attention to detail indicate a careful approach to prevent contamination or damage.


The final part involves utilizing a syringe to introduce liquid (likely calibration solution or cleaning solution) into the mass spectrometer.  This is done with precision and care inside a fume hood or a safety enclosure to handle potentially volatile solvents.


Based on my extensive experience in proteomics and mass spectrometry, the video demonstrates the procedure of removing and reattaching a nano-electrospray source on a Bruker timsTOF coupled with an Evosep system. This procedure is a crucial aspect of maintaining the instrument and ensuring accurate measurements.

**protocol number: 6**


{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


As Professor Matthias Mann, I have reviewed the provided video. The video shows a computer screen displaying the Bruker HyStar software.  The software is actively monitoring a mass spectrometry experiment, specifically what appears to be a liquid chromatography-mass spectrometry (LC-MS) run.  The display shows a chromatogram, a total ion chromatogram (TIC) that is tracking the total ion intensity over time, indicating the separation process is ongoing and that peptides or other analytes are eluting from the LC column. There is also a table of detected peaks with their respective mass-to-charge ratios (m/z), retention times, and intensities. The data suggests a data-dependent acquisition (DDA) or data-independent acquisition (DIA) method is being used, allowing for the identification and quantification of peptides or other molecules in the sample.

The software interface shows parameters such as scan range, ion mobility settings, and other instrument settings, all typical for a high-resolution mass spectrometry experiment. The video doesn't show any physical manipulation of equipment but instead focuses on the data acquisition and analysis phase of the experiment.  The parameters being adjusted on screen seem related to the optimization of the data acquisition process. The user is actively monitoring the instrument's performance, indicating a step in the experiment where the researcher is verifying the quality of the data acquisition in real-time.

Based on my expertise in proteomics and mass spectrometry, I believe the procedure shown is most likely the **data acquisition and monitoring stage of an LC-MS experiment** using the HyStar software for control and data analysis.  No sample handling or equipment changes are visible in this video; instead, it shows the software display during a live experiment.


**Protocol number: 1**


{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '1'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


As Professor Matthias Mann, I have reviewed the provided video. The video shows a laboratory procedure involving the preparation of an Evosep One system for liquid chromatography-mass spectrometry (LC-MS) analysis. 


Here's a detailed description of my observations:

The video focuses on a benchtop Evosep One LC system. The system is equipped with a 96-well plate holder designed to accommodate Evotip boxes. 


The actions performed include:

1. **Placement of Evotip boxes:** Two boxes of Evotips are placed onto the Evosep One.  One box is labelled and appears to contain samples while the second is labelled as "Blanks."  The boxes are meticulously positioned in designated slots (S1 and S3, presumably).

2. **Evotip inspection and handling:** The researcher uses forceps to carefully inspect individual Evotips from a separate, labeled box. The selection criteria for these tips appear related to their hydration status (rejecting tips which seem dry).  The selected Evotips are then manually transferred to the designated "Sample" box on the LC system.

3. **Documentation:** The hand-written labels on the boxes suggest a system for sample tracking and identification, a crucial aspect of any well-controlled experiment.


The materials handled are:

*   **Evotips:**  Single-use stage tips for sample preparation and injection.
*   **Evotip boxes:**  Specifically designed 96-well boxes to hold the Evotips.
*   **Forceps:** Used for handling individual Evotips.


The notable techniques are:

*   **Manual handling of Evotips:** Highlighting the importance of careful, contamination-free transfer of samples.
*   **Visual inspection of Evotips:** This indicates a quality control step to ensure proper sample preparation before LC-MS analysis.


**Based on my analysis, Protocol number: 2 ("Placing Evotips in Evotip Boxes on the Evosep One System") most accurately describes the laboratory procedure shown in the video.** The video does not show any steps related to the software operations (HyStar or TimsControl) or column connections/disconnections, which are prominent features of the other protocols. The focus is entirely on the physical placement and inspection of Evotips, and this aligns precisely with Protocol 2's objective.


Output: Protocol number: 2



{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


As Professor Matthias Mann, I have reviewed the provided video.  The video depicts a laboratory technician, wearing black nitrile gloves, preparing and loading  Evotip boxes into a centrifuge.

Specifically, the video shows the following:

1. **Equipment:** The video shows several pieces of equipment.  Most prominently, a Thermo Scientific Multifuge X Pro Series centrifuge is seen at the end of the procedure. The technician also uses a multichannel pipette to handle the Evotips.  There are also multiple boxes of Evotips visible, along with a rack for standard microcentrifuge tubes.

2. **Actions Performed:** The technician begins by carefully taking two boxes labeled "Evotips" and "Blank for injection" and holding them up. Next, the technician balances the boxes against each other (possibly visually or by weight), then proceeds to carefully place the boxes in the centrifuge. The centrifuge is then programmed via its touchscreen interface, with the speed and duration clearly visible at 800g for 1 minute.  The centrifuge is then started via the touchscreen.  The video ends with the centrifuge running.

3. **Materials Handled:** The primary materials are the Evotip boxes, pre-filled with what appears to be liquid. One box is labeled "Evotips" and the other "Blank for injection".  The blanks are likely empty or containing a wash solution.

4. **Notable Techniques:** The careful handling of the Evotip boxes suggests that the contents are sensitive.  The balancing of the boxes in the centrifuge is a critical step to ensure even centrifugation and prevent damage or uneven distribution of contents.  The use of a multichannel pipette (implied, not explicitly shown) indicates a high-throughput application.


Based on my observations, the protocol most accurately describing the laboratory procedure in the video is **protocol number: 10**. This protocol specifically details the procedure for spinning Evotips in a centrifuge, matching the actions and materials shown in the video precisely. The centrifugation parameters (800g for 1 minute) are also consistent with common practice in proteomics sample preparation.



{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 65.52%


In [21]:
example_video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/DisconnectingColumn_docuWithoutStandby_2.MP4"
example_video_uri = video_to_protocol.upload_video_to_gcs(example_video_path, bucket, "protocol_finder_2")

example_video_input = [
    "EXAMPLE SECTION:",
    "Eample Video:",
    Part.from_uri(example_video_uri, mime_type="video/mp4"),
    "Example Response:",
    """
    The researcher in the video is using following equipment: 
    - IonOpticks Column
    - timsTOF Mass Spectrometer
        - Equipped with UltraSource ion source
    - Evosep One LC System
        - with sample line
    - NanoViper Adapter (black)
        - Pliers

    Actions of the researcher in the video:
    - Opens the lid of a column oven.
    - Loosened the securing screw at the sample line - column connection. Lifts it up and moved the metal grounding screw away from the column-sample line connection.
    - Attaches the NanoViper adapter to the sample line for easier handling.
    - Helds the column fitting with pliers for easier handling.
    - Unscrews the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.
    - Positions the sample line ensuring it formed a gentle curve. Placs it over the transparent bumper of the Evosep.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: No content overlap. 
    - Protocol 2: No content overlap. 
    - Protocol 3: Overlap in content 95% & correct order of steps 
    - Protocol 4: No content overlap. 
    - Protocol 5: No content overlap. 
    - Protocol 6: Overlap in content 50%, but different order of steps.
    - Protocol 7: Overlap in content 20%, but different order of steps.
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 3 since content has the highest overlap and the order of steps matches best.
    END OF EXAMPLE SECTION
    """,
]
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    Your task is to analyze a video showing a laboratory procedure and identify which protocol from a provided list accurately describes the step being performed.
    
    I will first provide you with an EXAMPLE (video, and response). 
"""]
system_prompt_2 = ["""
    FORMATTING INSTRUCTIONS:
    1. Always begin your analysis by listing the equipment observed in the video using bullet points.
    2. Then describe the actions performed in the video using bullet points.
    3. Compare the video content with each protocol using this exact format:
    - Protocol 1: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    - Protocol 2: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    [Continue for all protocols]
    4. Conclude with a clear statement using this exact format:
    "Protocol number: X" where X is the number of the best matching protocol.
    5. If no protocol matches, explicitly state "Protocol number: None".
    6. Do not include any explanatory text after stating the protocol number.
                   
    YOUR TASK SECTION:
    Now, I provide you with the actuall 'lab video'. Follow the thought process in the example to analyze the video and match it to the selection of protocols.
"""]
protocol_intro= ["Protocols:"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]

    order = [system_prompt, example_video_input, system_prompt_2, video_parts[video]["part"], protocol_intro, protocol_input]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


The researcher in the video is using the following equipment:

- Evosep One LC system
- Bruker timsTOF mass spectrometer
- syringe
- nitrile gloves


Actions of the researcher in the video:

- The researcher approaches the timsTOF mass spectrometer.
- Removes a component from the timsTOF.
- Puts on nitrile gloves.
- Removes a second component from the timsTOF.
- Retrieves a syringe from a cabinet.
- Returns to the timsTOF and uses the syringe to perform some manipulation on the machine's components.
- Places the components back into the timsTOF.


- Protocol 1: 0%
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 20% & but different order of steps
- Protocol 8: 0%
- Protocol 9: 20% & but different order of steps
- Protocol 10: 0%

Protocol number: 7

{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '7'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


As Professor Matthias Mann, I have analyzed the provided video.


The video shows a computer screen displaying data from a mass spectrometry experiment.  The software interface is clearly shown, along with a chromatogram and a list of identified peptides. There is no manipulation of physical equipment.

- Equipment observed: Computer, software interface (likely Bruker Hystar).

- Actions performed: Monitoring of an ongoing mass spectrometry analysis.  The software displays the results in real-time, showing peaks corresponding to various peptides. No changes are being made to the experiment's parameters or setup.


- Protocol 1: 0%
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 0%


Protocol number: None


{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': None, 'message': 'No protocol number found in observation'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '7'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


Here's an analysis of the provided lab video following the specified format:

**Equipment Observed:**

* Evosep One LC System
* Evotip boxes
* Multichannel pipette
* Tips for multichannel pipette


**Actions Performed:**

* The researcher, wearing black nitrile gloves, retrieves two full boxes of Evotips from a drawer.
* Two boxes of Evotips are placed onto the Evosep One LC system.
* A third, partially filled Evotip box is also present.
* Using a multichannel pipette, the researcher transfers Evotips from the partially filled box to the boxes already on the instrument.


**Protocol Comparison:**

- Protocol 1: 0%
- Protocol 2: 90% & correct order of steps
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 0%


**Protocol number: 2**


{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Here's an analysis of the provided lab video, following the specified format:

**Equipment Observed:**

* Multifuge X Pro Series Centrifuge (Thermo Scientific)
*  Boxes of Evotips
* Multichannel pipette

**Actions Performed:**

* The researcher retrieves two boxes of Evotips.
* The researcher places the boxes into the centrifuge.
* The researcher programs the centrifuge for 800g for 1 minute.
* The researcher starts the centrifuge.


**Protocol Comparison:**

- Protocol 1: 0%
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 95% & correct order of steps


**Protocol number: 10**



{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 62.07%


In [22]:
example_video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/DisconnectingColumn_docuWithoutStandby_2.MP4"
example_video_uri = video_to_protocol.upload_video_to_gcs(example_video_path, bucket, "protocol_finder_2")
example_video_path_2 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/QueueSamples_docuCorrect_2.mov"
example_video_uri_2 = video_to_protocol.upload_video_to_gcs(example_video_path_2, bucket, "protocol_finder_2")

example_video_input = [
    "EXAMPLE SECTION:",
    "Eample Video 1:",
    Part.from_uri(example_video_uri, mime_type="video/mp4"),
    "Example Response 1:",
    """
    The researcher in the video is using following equipment: 
    - IonOpticks Column
    - timsTOF Mass Spectrometer
        - Equipped with UltraSource ion source
    - Evosep One LC System
        - with sample line
    - NanoViper Adapter (black)
        - Pliers

    Actions of the researcher in the video:
    - Opens the lid of a column oven.
    - Loosened the securing screw at the sample line - column connection. Lifts it up and moved the metal grounding screw away from the column-sample line connection.
    - Attaches the NanoViper adapter to the sample line for easier handling.
    - Helds the column fitting with pliers for easier handling.
    - Unscrews the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.
    - Positions the sample line ensuring it formed a gentle curve. Placs it over the transparent bumper of the Evosep.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: No content overlap. 
    - Protocol 2: No content overlap. 
    - Protocol 3: Overlap in content 95% & correct order of steps 
    - Protocol 4: No content overlap. 
    - Protocol 5: No content overlap. 
    - Protocol 6: Overlap in content 50%, but different order of steps.
    - Protocol 7: Overlap in content 20%, but different order of steps.
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 3 since it's content has the highest overlap and the order of steps matches best.""",

    "Eample Video 2:",
    Part.from_uri(example_video_uri_2, mime_type="video/mov"),
    "Example Response 2:",
    """
    The researcher in the video is using following equipment: 
    - software: HyStar 6.0

    Actions of the researcher in the video:
    - Navigates to the 'Acquisition' tab in Hystar software.
    - Selects an existing sample table by pressing the arrow down button when hovering over the sample table name.
    - Copies existing sample table entries for modification.
    - Adjusts the sample ID following a specific naming pattern.
    - Queues analysis runs in sequence: one blank, three dda-PASEF runs, three dia-PASEF runs, and a final blank.
    - Verifies column autocompletion settings through right-click menu, ensuring proper tray configuration.
    - Specifies Evotip positions by selecting the first position and using drag-to-fill functionality for remaining samples.
    - Checks the storage path folder for raw files.
    - Verifies the separation method settings.
    - Confirms injection method is set to 'standard'.
    - Adjusts dda-PASEF and dia-PASEF maintenance methods to match sample IDs.
    - Stops the idle flow on the Evosep system through right-click menu option.
    - Saves the updated sample table.
    - Uploads sample conditions by right-clicking the top row to verify proper loading of LC and MS methods.
    - Initiates the sequence by pressing the 'start' and 'start sequence' buttons.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: Overlap in content 100% & correct order of steps 
    - Protocol 2: No content overlap. 
    - Protocol 3: No content overlap. 
    - Protocol 4: No content overlap. 
    - Protocol 5: Overlap in content 5%.  
    - Protocol 6: No content overlap. 
    - Protocol 7: No content overlap. 
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 1 since it's content has the highest overlap and the order of steps matches best.

    END OF EXAMPLE SECTION
    """,
]
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    Your task is to analyze a video showing a laboratory procedure and identify which protocol from a provided list accurately describes the step being performed.
    
    I will first provide you with an EXAMPLE (video, and response). 
"""]
system_prompt_2 = ["""
    FORMATTING INSTRUCTIONS:
    1. Always begin your analysis by listing the equipment observed in the video using bullet points.
    2. Then describe the actions performed in the video using bullet points.
    3. Compare the video content with each protocol using this exact format:
    - Protocol 1: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    - Protocol 2: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    [Continue for all protocols]
    4. Conclude with a clear statement using this exact format:
    "Protocol number: X" where X is the number of the best matching protocol.
    5. If no protocol matches, explicitly state "Protocol number: None".
    6. Do not include any explanatory text after stating the protocol number.
                   
    YOUR TASK SECTION:
    Now, I provide you with the actuall 'lab video'. Follow the thought process in the example to analyze the video and match it to the selection of protocols.
"""]
protocol_intro= ["Protocols:"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]

    order = [system_prompt, example_video_input, system_prompt_2, video_parts[video]["part"], protocol_intro, protocol_input]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-flash-002", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


Okay, let's analyze this video.  My expertise in proteomics and mass spectrometry will help me identify the specific protocol.


**Equipment Observed:**

*   IonOpticks Column
*   timsTOF Mass Spectrometer (Bruker)
    *   Equipped with UltraSource ion source
*   Evosep One LC System
    *   with sample line
*   NanoViper Adapter (appears to be metallic)
*   Pliers
*   Screwdriver
*   Black nitrile gloves


**Actions Performed:**

*   The researcher puts on black nitrile gloves.
*   The researcher removes the top cover of the timsTOF's UltraSource.
*   The researcher removes a metallic component (likely part of the UltraSource) from the timsTOF.
*   The researcher carefully handles the removed component.
*   The researcher appears to clean or inspect the removed component.
*   The researcher re-inserts the metallic component into the timsTOF's UltraSource.
*   The researcher connects something to the metallic component using a small screwdriver.
*   The researcher reassembles the top cover of the timsTOF's UltraSource.


**Protocol Comparison:**

- Protocol 1: 0%
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 20% but different order of steps
- Protocol 7: 90% & correct order of steps
- Protocol 8: 0%
- Protocol 9: 80% & correct order of steps
- Protocol 10: 0%


**Protocol number: 7**



{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': '7'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


Professor Matthias Mann's Analysis of Lab Video:

1. Equipment Observed:

- Computer monitor displaying Bruker HyStar software.


2. Actions Performed:

- The software displays a chromatogram and mass spectrum.
- The software shows a sample queue is being reviewed and modified.  Specific parameters, such as sample IDs, are being adjusted within the sample table.
- The researcher appears to be checking and adjusting method settings within the software.
- The researcher appears to be confirming the sample tray configuration.
- The researcher initiates the sample queue.


3. Protocol Comparison:

- Protocol 1: 90% & correct order of steps
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 0%


4. Best Matching Protocol:

Protocol number: 1



{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '1'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '6'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


Here's an analysis of the provided lab video, following the specified format:

**Equipment Observed:**

* Evosep One LC System
* EvoTip boxes
* Multichannel pipette
* Tweezers

**Actions Performed:**

* The researcher retrieves two full EvoTip boxes from storage.
* The researcher places the EvoTip boxes onto the Evosep One deck.
* The researcher retrieves a third, empty EvoTip box from storage.
* The researcher places the empty EvoTip box onto the Evosep One deck.
* The researcher uses a multichannel pipette to transfer EvoTips from the full boxes to the empty box.


**Protocol Comparison:**

- Protocol 1: 0%
- Protocol 2: 90% & correct order of steps
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 0%

**Protocol number: 2**



{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


Here's my analysis of the provided lab video, following the specified format:

**Equipment Observed:**

- Multifuge X Pro Series Centrifuge (Thermo Scientific)
- Evotip boxes
- Multichannel pipette


**Actions Performed:**

- The researcher retrieves two Evotip boxes.
- The researcher places the boxes into the centrifuge.
- The researcher sets the centrifuge parameters (speed and time) via the touchscreen interface.
- The researcher initiates the centrifuge run.


**Protocol Comparison:**

- Protocol 1: 0%
- Protocol 2: 90% & correct order of steps
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 0%
- Protocol 6: 0%
- Protocol 7: 0%
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 100% & correct order of steps


**Protocol number: 10**



{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 51.72%


In [30]:
example_video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/DisconnectingColumn_docuWithoutStandby_2.MP4"
example_video_uri = video_to_protocol.upload_video_to_gcs(example_video_path, bucket, "protocol_finder_2")
example_video_path_2 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/QueueSamples_docuCorrect_2.mov"
example_video_uri_2 = video_to_protocol.upload_video_to_gcs(example_video_path_2, bucket, "protocol_finder_2")

example_video_input = [
    "EXAMPLE SECTION:",
    "Eample Video 1:",
    Part.from_uri(example_video_uri, mime_type="video/mp4"),
    "Example Response 1:",
    """
    The researcher in the video is using following equipment: 
    - IonOpticks Column
    - timsTOF Mass Spectrometer
        - Equipped with UltraSource ion source
    - Evosep One LC System
        - with sample line
    - NanoViper Adapter (black)
        - Pliers

    Actions of the researcher in the video:
    - Opens the lid of a column oven.
    - Loosened the securing screw at the sample line - column connection. Lifts it up and moved the metal grounding screw away from the column-sample line connection.
    - Attaches the NanoViper adapter to the sample line for easier handling.
    - Helds the column fitting with pliers for easier handling.
    - Unscrews the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.
    - Positions the sample line ensuring it formed a gentle curve. Placs it over the transparent bumper of the Evosep.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: No content overlap. 
    - Protocol 2: No content overlap. 
    - Protocol 3: Overlap in content 95% & correct order of steps 
    - Protocol 4: No content overlap. 
    - Protocol 5: No content overlap. 
    - Protocol 6: Overlap in content 50%, but different order of steps.
    - Protocol 7: Overlap in content 20%, but different order of steps.
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 3 since it's content has the highest overlap and the order of steps matches best.""",

    "Eample Video 2:",
    Part.from_uri(example_video_uri_2, mime_type="video/mov"),
    "Example Response 2:",
    """
    The researcher in the video is using following equipment: 
    - software: HyStar 6.0

    Actions of the researcher in the video:
    - Navigates to the 'Acquisition' tab in Hystar software.
    - Selects an existing sample table by pressing the arrow down button when hovering over the sample table name.
    - Copies existing sample table entries for modification.
    - Adjusts the sample ID following a specific naming pattern.
    - Queues analysis runs in sequence: one blank, three dda-PASEF runs, three dia-PASEF runs, and a final blank.
    - Verifies column autocompletion settings through right-click menu, ensuring proper tray configuration.
    - Specifies Evotip positions by selecting the first position and using drag-to-fill functionality for remaining samples.
    - Checks the storage path folder for raw files.
    - Verifies the separation method settings.
    - Confirms injection method is set to 'standard'.
    - Adjusts dda-PASEF and dia-PASEF maintenance methods to match sample IDs.
    - Stops the idle flow on the Evosep system through right-click menu option.
    - Saves the updated sample table.
    - Uploads sample conditions by right-clicking the top row to verify proper loading of LC and MS methods.
    - Initiates the sequence by pressing the 'start' and 'start sequence' buttons.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: Overlap in content 100% & correct order of steps 
    - Protocol 2: No content overlap. 
    - Protocol 3: No content overlap. 
    - Protocol 4: No content overlap. 
    - Protocol 5: Overlap in content 5%.  
    - Protocol 6: No content overlap. 
    - Protocol 7: No content overlap. 
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 1 since it's content has the highest overlap and the order of steps matches best.

    END OF EXAMPLE SECTION
    """,
]
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    Your task is to analyze a video showing a laboratory procedure and identify which protocol from a provided list accurately describes the step being performed.
    
    I will first provide you with an EXAMPLE (video, and response). 
"""]
system_prompt_2 = ["""
    FORMATTING INSTRUCTIONS:
    1. Always begin your analysis by listing the equipment observed in the video using bullet points.
    2. Then describe the actions performed in the video using bullet points.
    3. Compare the video content with each protocol using this exact format:
    - Protocol 1: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    - Protocol 2: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    [Continue for all protocols]
    4. Conclude with a clear statement using this exact format:
    "Protocol number: X" where X is the number of the best matching protocol.
    5. If no protocol matches, explicitly state "Protocol number: None".
    6. Do not include any explanatory text after stating the protocol number.
                   
    YOUR TASK SECTION:
    Now, I provide you with the actuall 'lab video'. Follow the thought process in the example to analyze the video and match it to the selection of protocols.
"""]
protocol_intro= ["Protocols:"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]

    order = [system_prompt, example_video_input, system_prompt_2, video_parts[video]["part"], protocol_intro, protocol_input]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-pro-001", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


- timsTOF SCP Mass Spectrometer
    - Equipped with UltraSource ion source
- Evosep One LC System
- IonOpticks Column
- Metal grounding screw 
- Tweezers
- Laboratory bench
- Gloves
- Syringe
- Formic acid
- Acetonitrile
- Calibration mix

Actions of the researcher in the video:
- Opens the lid of the timsTOF UltraSource.
- Disconnects the UltraSource ion source from the mass spectrometer.
- Places the UltraSource onto a lab bench.
- Puts on gloves.
- Disassembles the UltraSource by removing the ceramic insulator.
- Reassembles the UltraSource by reinstalling the ceramic insulator and attaching it back to the timsTOF mass spectrometer.
- Closes the UltraSource lid.
- Uses a syringe to flush the emitter with a mixture of formic acid and acetonitrile.
- Verifies the calibration using a calibration mix.

- Protocol 1: 0% & different order of steps
- Protocol 2: 0%
- Protocol 3: 0%
- Protocol 4: 0%
- Protocol 5: 15% & different order of steps
- Protocol 6: 0%
- Protocol 7: 20% & different order of steps
- Protocol 8: 0%
- Protocol 9: 0%
- Protocol 10: 0%

Protocol number: None 


{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': None, 'message': 'No protocol number found in observation'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


- Bruker Daltonics timsTOF Ultra Mass Spectrometer
  - Equipped with ESI source
  - UltraSource ion source ready to attach
- HP Eye Ease monitor displaying Hystar software
- IonOpticks Column
- Evosep One LC System with sample line
- NanoViper Adapter (black)
- Pliers

- Navigates through Hystar software, specifically the 'Acquisition' tab.
- Focuses on tuning and calibration of the TIMS (Trapped Ion Mobility Spectrometry) device.
- Adjusts settings related to MS (Mass Spectrometry) mode, scan range, and ion mobility.
- Sets the TIMS mode to 'Custom' and adjusts parameters like ramp time and duty cycle.
- Verifies and potentially modifies the reference list for calibration, selecting specific calibrants for tuning.
- Confirms the calibration status and score in the 'Calibration' tab.

- Protocol 1: Overlap in content 95% & correct order of steps
- Protocol 2: No content overlap.
- Protocol 3: No content overlap.
- Protocol 4: No content overlap.
- Protocol 5: Overlap in content 95% & correct order of steps
- Protocol 6: No content overlap.
- Protocol 7: No content overlap.
- Protocol 8: No content overlap.
- Protocol 9: No content overlap.
- Protocol 10: No content overlap.

Protocol number: 5 


{'matches': True, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '5'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '7'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


- Evosep One LC System
  - with two Evotip boxes
  - with Evotips
- Tweezers
- Pipette

Actions of the researcher in the video:
- Places two Evotip boxes on the Evosep system.
- Places Evotips into an Evotip box using tweezers. 
- Notes the positions of the Evotips.

Compare the video content with the content and order of steps in each of the protocols. 
List your answer as follows:
- Protocol 1: Overlap in content 20%, but different order of steps.
- Protocol 2: Overlap in content 100% & correct order of steps.
- Protocol 3: No content overlap. 
- Protocol 4: No content overlap.
- Protocol 5: No content overlap. 
- Protocol 6: Overlap in content 20%, but different order of steps.
- Protocol 7: No content overlap. 
- Protocol 8: No content overlap. 
- Protocol 9: No content overlap. 
- Protocol 10: No content overlap. 

Protocol number: 2 


{'matches': True, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '2'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


- Evotip boxes
- Evotips
- Multifuge X Pro Series Centrifuge
- Multichannel pipette

Actions of the researcher in the video:
- Balances Evotip boxes by placing an empty Evotip box on the multichannel pipette.
- Places Evotips in a centrifuge.
- Sets the speed of the centrifuge to 800g.
- Sets the time of the centrifuge to 1 minute.
- Starts the centrifuge.

Compare the video content with the content and order of steps in each of the protocols. 
List your answer as follows:
- Protocol 1: Overlap in content 10%, but different order of steps.
- Protocol 2: Overlap in content 50%, but different order of steps.
- Protocol 3: No content overlap. 
- Protocol 4: No content overlap. 
- Protocol 5: No content overlap. 
- Protocol 6: No content overlap. 
- Protocol 7: No content overlap. 
- Protocol 8: No content overlap. 
- Protocol 9: No content overlap. 
- Protocol 10: Overlap in content 90% & correct order of steps.

Protocol number: 10


{'matches': True, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '10'}
Success rate: 72.41%


In [31]:
example_video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/DisconnectingColumn_docuWithoutStandby_2.MP4"
example_video_uri = video_to_protocol.upload_video_to_gcs(example_video_path, bucket, "protocol_finder_2")
example_video_path_2 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/unused/QueueSamples_docuCorrect_2.mov"
example_video_uri_2 = video_to_protocol.upload_video_to_gcs(example_video_path_2, bucket, "protocol_finder_2")

example_video_input = [
    "EXAMPLE SECTION:",
    "Eample Video 1:",
    Part.from_uri(example_video_uri, mime_type="video/mp4"),
    "Example Response 1:",
    """
    The researcher in the video is using following equipment: 
    - IonOpticks Column
    - timsTOF Mass Spectrometer
        - Equipped with UltraSource ion source
    - Evosep One LC System
        - with sample line
    - NanoViper Adapter (black)
        - Pliers

    Actions of the researcher in the video:
    - Opens the lid of a column oven.
    - Loosened the securing screw at the sample line - column connection. Lifts it up and moved the metal grounding screw away from the column-sample line connection.
    - Attaches the NanoViper adapter to the sample line for easier handling.
    - Helds the column fitting with pliers for easier handling.
    - Unscrews the NanoViper connector to detach the sample line from the IonOpticks column by turning it counter clock-wise.
    - Positions the sample line ensuring it formed a gentle curve. Placs it over the transparent bumper of the Evosep.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: No content overlap. 
    - Protocol 2: No content overlap. 
    - Protocol 3: Overlap in content 95% & correct order of steps 
    - Protocol 4: No content overlap. 
    - Protocol 5: No content overlap. 
    - Protocol 6: Overlap in content 50%, but different order of steps.
    - Protocol 7: Overlap in content 20%, but different order of steps.
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 3 since it's content has the highest overlap and the order of steps matches best.""",

    "Eample Video 2:",
    Part.from_uri(example_video_uri_2, mime_type="video/mov"),
    "Example Response 2:",
    """
    The researcher in the video is using following equipment: 
    - software: HyStar 6.0

    Actions of the researcher in the video:
    - Navigates to the 'Acquisition' tab in Hystar software.
    - Selects an existing sample table by pressing the arrow down button when hovering over the sample table name.
    - Copies existing sample table entries for modification.
    - Adjusts the sample ID following a specific naming pattern.
    - Queues analysis runs in sequence: one blank, three dda-PASEF runs, three dia-PASEF runs, and a final blank.
    - Verifies column autocompletion settings through right-click menu, ensuring proper tray configuration.
    - Specifies Evotip positions by selecting the first position and using drag-to-fill functionality for remaining samples.
    - Checks the storage path folder for raw files.
    - Verifies the separation method settings.
    - Confirms injection method is set to 'standard'.
    - Adjusts dda-PASEF and dia-PASEF maintenance methods to match sample IDs.
    - Stops the idle flow on the Evosep system through right-click menu option.
    - Saves the updated sample table.
    - Uploads sample conditions by right-clicking the top row to verify proper loading of LC and MS methods.
    - Initiates the sequence by pressing the 'start' and 'start sequence' buttons.
    
    Compare the video content with the content and order of steps in each of the protocols. 
    List your answer as follows:
    - Protocol 1: Overlap in content 100% & correct order of steps 
    - Protocol 2: No content overlap. 
    - Protocol 3: No content overlap. 
    - Protocol 4: No content overlap. 
    - Protocol 5: Overlap in content 5%.  
    - Protocol 6: No content overlap. 
    - Protocol 7: No content overlap. 
    - Protocol 8: No content overlap. 
    - Protocol 9: No content overlap. 
    - Protocol 10: No content overlap. 

    Which protocol matches best?
    Protocol number: 1 since it's content has the highest overlap and the order of steps matches best.

    END OF EXAMPLE SECTION
    """,
]
system_prompt = ["""
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    You have planty of documents that youonly use to look how the equipment is called exactly."""]

# Upload knowledge files to Google Cloud Storage
folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base"
subfolder_in_bucket = "knowledge"

knowledge_uris = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(
        (".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".tif", ".pdf")
    ):
        path = Path(folder_path) / filename
        try:
            file_uri = video_to_protocol.upload_video_to_gcs(
                path, bucket, subfolder_in_bucket
            )
            knowledge_uris.append(file_uri)
        except OSError as e:
            print(f"Error processing {filename}: {e}")
MIME_TYPES = {
    ".pdf": "application/pdf",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".png": "image/png",
}
contents = []
for file_path in knowledge_uris:
        path_obj = Path(file_path)
        file_ext = path_obj.suffix.lower()

        if file_ext in MIME_TYPES:
            mime_type = MIME_TYPES[file_ext]
            contents.append(Part.from_uri(file_path, mime_type=mime_type))


                 
system_prompt_1 = ["""
    Your task is to analyze a video showing a laboratory procedure and identify which protocol from a provided list accurately describes the step being performed.
    
    I will first provide you with an EXAMPLE (video, and response). 
"""]
system_prompt_2 = ["""
    FORMATTING INSTRUCTIONS:
    1. Always begin your analysis by listing the equipment observed in the video using bullet points.
    2. Then describe the actions performed in the video using bullet points.
    3. Compare the video content with each protocol using this exact format:
    - Protocol 1: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    - Protocol 2: [Overlap percentage]% [& correct order of steps OR but different order of steps]
    [Continue for all protocols]
    4. Conclude with a clear statement using this exact format:
    "Protocol number: X" where X is the number of the best matching protocol.
    5. If no protocol matches, explicitly state "Protocol number: None".
    6. Do not include any explanatory text after stating the protocol number.
                   
    YOUR TASK SECTION:
    Now, I provide you with the actuall 'lab video'. Follow the thought process in the example to analyze the video and match it to the selection of protocols.
"""]
protocol_intro= ["Protocols:"]

csv_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/Link_protocol_lab_video.csv"

validation_list = list()
for video in video_parts: 
    print(f"Processing video: {video}")
    video_path = video_parts[video]["path"]

    order = [system_prompt, contents, system_prompt_1, example_video_input, system_prompt_2, video_parts[video]["part"], protocol_intro, protocol_input]
    model_input = list()
    for input in order:
        model_input.extend(input)

    result = identify_protocol_from_video(
        model_input,
        model_name="gemini-1.5-pro-001", 
        temperature=0.9
    )

    # print(result["usage_metadata"])
    print(result["video_path"])
    display(result["markdown_observation"]) 

    result_validation = validate_protocol_number(video_path, csv_path, result["observation"])
    print(result_validation)
    validation_list.append(result_validation["matches"])

success_ratio = sum(validation_list) / len(validation_list) if len(validation_list) > 0 else 0
print(f"Success rate: {success_ratio:.2%}")

Processing video: UltraSourceToESIsource_docuForgotN2Line.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/UltraSourceToESIsource_docuForgotN2Line.MP4


- IonOpticks column
- timsTOF SCP mass spectrometer
    - equipped with an UltraSource ion source
- Evosep One LC system
- Tweezers
- Wrench
- Screwdriver
- Acetonitrile
- Water
- Pliers
- Kimwipe
- Syringe
- Lint-free tissue
- Gloves
- Fume hood
- Nitrogen gas

- Disconnects the IonOpticks column from the Evosep One LC System
- Disassembles the UltraSource from the timsTOF SCP system
- Places the UltraSource on a work bench
- Disassembles the spray shield from the UltraSource
- Cleans the spray shield with water and acetonitrile
- Inspects the capillary
- Cleans the capillary by flushing it with water and acetonitrile
- Reassembles the spray shield to the UltraSource
- Reattaches the UltraSource to the timsTOF SCP system

- Protocol 1: 0% & but different order of steps
- Protocol 2: 0% & but different order of steps
- Protocol 3: 0% & but different order of steps
- Protocol 4: 0% & but different order of steps
- Protocol 5: 0% & but different order of steps
- Protocol 6: 0% & but different order of steps
- Protocol 7: 0% & but different order of steps
- Protocol 8: 0% & but different order of steps
- Protocol 9: 0% & but different order of steps
- Protocol 10: 0% & but different order of steps

Protocol number: None 


{'matches': False, 'video': 'UltraSourceToESIsource_docuForgotN2Line.MP4', 'expected_protocol': '7', 'found_protocol': None, 'message': 'No protocol number found in observation'}
Processing video: TimsCalibration_docuCorrect_camera.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuCorrect_camera.MP4


- Equipment:
    - Computer with timsControl software
    - timsTOF mass spectrometer
    - Tuning Mix ESI
    - Syringe pump
    - Vial
    
- Actions:
    - The researcher checks the TIMS settings in timsControl software.
    - The researcher confirms the flow rate, ramp time, accumulation time and duty cycle settings.
    - The researcher adjusts MS averaging value to 1.
    - The researcher navigates to the 'Calibration' tab and selects 'Mobility'.
    - The researcher clicks the 'Calibration Mode' drop-down menu and chooses 'Linear'.
    - The researcher clicks 'Calibrate'.

- Protocol 1: 100% & correct order of steps
- Protocol 2: 0% & but different order of steps
- Protocol 3: 0% & but different order of steps
- Protocol 4: 0% & but different order of steps
- Protocol 5: 30% & but different order of steps
- Protocol 6: 0% & but different order of steps
- Protocol 7: 0% & but different order of steps
- Protocol 8: 0% & but different order of steps
- Protocol 9: 0% & but different order of steps
- Protocol 10: 10% & but different order of steps

Protocol number: 1


{'matches': False, 'video': 'TimsCalibration_docuCorrect_camera.MP4', 'expected_protocol': '5', 'found_protocol': '1'}
Processing video: TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/TimsCalibration_docuNotAllClicksVisibleOnVideo.MP4


...

{'matches': True, 'video': 'UltraSourceToESIsource_docuForgotCapillaryCap.MP4', 'expected_protocol': '7', 'found_protocol': '7'}
Processing video: PlaceEvotips_docuLiquidNotChecked.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/PlaceEvotips_docuLiquidNotChecked.MP4


- **Equipment:**
    - Evosep One LC system
    - Evotip Pure tips
    - Yellow tip box

- **Actions:**
    - Loads the Evotip Pure tips into a yellow tip box.
    - Places the loaded tip box on the Evosep One LC system.

- **Protocol Analysis:**
    - Protocol 1: **100% overlap & correct order of steps.**
    - Protocol 2: **20% overlap but different order of steps.**
    - Protocol 3: **No content overlap.**
    - Protocol 4: **No content overlap.**
    - Protocol 5: **No content overlap.**
    - Protocol 6: **No content overlap.**
    - Protocol 7: **No content overlap.**
    - Protocol 8: **No content overlap.**
    - Protocol 9: **No content overlap.**
    - Protocol 10: **No content overlap.**

**Protocol number: 1**

{'matches': False, 'video': 'PlaceEvotips_docuLiquidNotChecked.MP4', 'expected_protocol': '2', 'found_protocol': '1'}
Processing video: Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4
/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4


- Equipment observed in the video:
    - Evotip holders with solvent reservoir
    - Two boxes of Evotips (one black, one transparent)
    - Tabletop centrifuge
    - Eppendorf multichannel pipette
    - Pipette tip box

- Actions performed in the researcher in the video:
    - Balances two Evotip boxes on a balance by adding one extra Evotip to the lighter box
    - Opens the lid of a tabletop centrifuge
    - Places both Evotip boxes inside the centrifuge, each in a separate holder, at opposite positions
    - Closes the centrifuge lid
    - Enters spin settings on the centrifuge's control panel
    - Starts the centrifuge

- Comparison with provided protocols:
    - Protocol 1: 5% & correct order of steps
    - Protocol 2: 95% & correct order of steps
    - Protocol 3: 0%
    - Protocol 4: 0%
    - Protocol 5: 20% & correct order of steps
    - Protocol 6: 0%
    - Protocol 7: 0%
    - Protocol 8: 0%
    - Protocol 9: 0%
    - Protocol 10: 95% & correct order of steps

Protocol number: 2 


{'matches': False, 'video': 'Evotip_docuWrongSpinInEvotipBox_wrongSpeed.MP4', 'expected_protocol': '10', 'found_protocol': '2'}
Success rate: 44.83%
